# Graphiti Agent

그래피티(Graphiti)는 AI 에이전트를 위해 설계된 시간 인식 지식 그래프 구축을 위한 파이썬 프레임워크입니다. 일괄 재계산 없이 지식 그래프에 대한 실시간 증분 업데이트를 가능하게 하여, 관계와 정보가 시간에 따라 진화하는 동적 환경에 적합합니다.

![](https://steemitimages.com/300x0/https://github.com/gongwon-nayeon/graphiti-agent-tutorial/raw/main/graphiti_agent.png)

참고: https://github.com/gongwon-nayeon/graphiti-agent-tutorial/blob/main/graphiti_agent.ipynb

이번 튜토리얼에서는 [Neo4j Sandbox](https://sandbox.neo4j.com/)를 사용합니다.
> Neo4j Sandbox란? **온라인 그래프 데이터베이스**로, 무료 클라우드 기반 Neo4j 인스턴스를 구축할 수 있기 떄문에 간편하게 리소스를 빌려 그래프DB를 실험 할 수 있습니다.

## 설치하기

필요한 패키지((graphiti, langgraph, langchain, 등)을 설치합니다.

In [1]:
%%capture --no-stderr
%pip install -qU langgraph langchain-core langchain-openai 
%pip install -qU graphiti-core

## 환경 설정

graphiti를 실행하는데 필요한 환경변수를 설정합니다. graphiti는 OpenAI를 사용하므로 OpenAI API Key가 필요합니다.

In [2]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langchain-academy"

neo4j_uri = os.environ.get("NEO4J_URI")
neo4j_user = os.environ.get("NEO4J_USERNAME")
neo4j_password = os.environ.get("NEO4J_PASSWORD")

if not neo4j_uri or not neo4j_user or not neo4j_password:
    raise ValueError("NEO4J_URI, NEO4J_USERNAME, and NEO4J_PASSWORD must be set")

In [3]:
import logging

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger(__name__)

## Graphiti 사용하기

In [ ]:
from graphiti_core import Graphiti

# Graphiti 초기화
graphiti = Graphiti(neo4j_uri, neo4j_user, neo4j_password)

await graphiti.build_indices_and_constraints()

2025-10-20 20:54:45 - neo4j.notifications - INFO - Received notification from DBMS server: <GqlStatusObject gql_status='00NA0', status_description="note: successful completion - index or constraint already exists. The command 'CREATE RANGE INDEX valid_at_episodic_index IF NOT EXISTS FOR (e:Episodic) ON (e.valid_at)' has no effect. The index or constraint specified by 'RANGE INDEX valid_at_episodic_index FOR (e:Episodic) ON (e.valid_at)' already exists.", position=None, raw_classification='SCHEMA', classification=<NotificationClassification.SCHEMA: 'SCHEMA'>, raw_severity='INFORMATION', severity=<NotificationSeverity.INFORMATION: 'INFORMATION'>, diagnostic_record={'_classification': 'SCHEMA', '_severity': 'INFORMATION', 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: 'CREATE INDEX valid_at_episodic_index IF NOT EXISTS FOR (n:Episodic) ON (n.valid_at)'
2025-10-20 20:54:45 - neo4j.notifications - INFO - Received notification from DBMS server: <GqlStatusObject gq

## 1. 에피소드 추가

에피소드는 Graphiti의 기본 단위입니다.

**에피소드 유형**은 다음과 같습니다.
- `text`: 비정형 택스트 데이터
- `message`: 대화형 메시지 형식
- `json`: 구조화된 데이터

In [5]:
from graphiti_core.nodes import EpisodeType

episodes = [
    {
        "type": EpisodeType.json,
        "name": "OpenAI",
        "content": {
            "name": "OpenAI",
            "founded": "2015-12-8",
            "founders": "Sam Altman",
            "type": "AI 연구 개발 기업",
            "official_website": "https://openai.com/",
        },
        "description": "article metadata",
    },
    {
        "type": EpisodeType.text,
        "name": "OpenAI의 기본 모델",
        "content": "2024년 5월 기준, OpenAI의 기본 모델은 GPT-4o입니다.",
        "description": "news article",
    },
]

In [7]:
# 에피소스 등록하기
import json
from datetime import datetime, timezone


for i, episode in enumerate(episodes):
    await graphiti.add_episode(
        name=episode["name"],
        episode_body=episode["content"]
        if isinstance(episode["content"], str)
        else json.dumps(episode["content"]),
        source=episode["type"],
        source_description=episode["description"],
        reference_time=datetime.now(timezone.utc),
    )
    print(f"Added episode: {episode['name']} ({episode['type'].value})")

2025-10-20 20:55:02 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-10-20 20:55:04 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-20 20:55:04 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-20 20:55:04 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-20 20:55:04 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-20 20:55:07 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-10-20 20:55:17 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-10-20 20:55:18 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-20 20:55:20 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025

Added episode: OpenAI (json)


2025-10-20 20:55:42 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-10-20 20:55:43 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-20 20:55:43 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-20 20:55:46 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-10-20 20:55:48 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-20 20:55:49 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-20 20:55:50 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-20 20:55:52 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-10-20 20:55:52 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025

Added episode: OpenAI의 기본 모델 (text)


새로운 에피소드 추가하기

In [ ]:
new_episodes = [
    {
        "name": "새로운 모델 출시",
        "content": "OpenAI의 현재 기본 모델은 8월 8일부로 GPT-5로 변경되었습니다."
        "GPT-5는 전반적으로 훨씬 더 스마트하며, 특히 수학, 코딩, 시각적 인식, 의료 분야의 학술적 및 인간 평가 벤치마크에서의 성과에서 확인할 수 있습니다. "
        "수학(도구 없이 AIME 2025에서 94.6%), 실제 코딩(SWEBench에서 74.9%, 에이더 폴리글롯에서 88%), 멀티모달 이해(MMU에서 84.2$), 의료(HealthBench Hard에서 46.2%) 전반에서 새롭게 최고 기록을 세웠으며 이러한 이점은 일상 사용에도 적용됩니다. ",
        "type": EpisodeType.text,
        "description": "news article",
    }
]

for i, episode in enumerate(new_episodes):
    await graphiti.add_episode(
        name=episode["name"],
        episode_body=episode["content"]
        if isinstance(episode["content"], str)
        else json.dumps(episode["content"]),
        source=episode["type"],
        source_description=episode["description"],
        reference_time=datetime.now(timezone.utc),
    )
    print(f"Added episode: {episode['name']} ({episode['type'].value})")

2025-10-20 20:57:05 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-10-20 20:57:05 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-20 20:57:05 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-20 20:57:05 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-20 20:57:05 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-20 20:57:05 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-20 20:57:05 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-20 20:57:05 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-20 20:57:06 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
20

Added episode: 새로운 모델 출시 (text)


## 2. 에피소드 검색하기

### (1) 기본 검색

In [ ]:
query = "OpenAI의 정보를 알려주세요."

results = await graphiti.search(query)

for result in results:
    print(f"UUID: {result.uuid}")
    print(f"Fact: {result.fact}")
    if hasattr(result, "valid_at") and result.valid_at:
        print(f"Valid from: {result.valid_at}")
    if hasattr(result, "invalid_at") and result.invalid_at:
        print(f"Valid until: {result.invalid_at}")
    print("-" * 50)

2025-10-20 21:10:51 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


UUID: 19301958-398a-4976-807f-9976251958dd
Fact: GPT-5가 OpenAI의 현재 기본 모델이다.
Valid from: 2025-10-20 11:56:59.540507+00:00
--------------------------------------------------
UUID: 12176a8f-c5dc-47c2-af33-b2157b4d8066
Fact: OpenAI is an AI 연구 개발 기업 (AI research and development company).
Valid from: 2025-10-19 09:23:22.008159+00:00
Valid until: 2025-10-19 09:24:03.380744+00:00
--------------------------------------------------
UUID: 87e7cbf8-d653-4e79-ab97-63782189b381
Fact: OpenAI의 기본 모델이 2025-08-08부로 변경되었다.
Valid from: 2025-08-08 00:00:00+00:00
Valid until: 2025-10-19 09:23:22.008159+00:00
--------------------------------------------------
UUID: 5ac5606a-d491-410e-ab7b-71f64755fc06
Fact: 2024년 5월 기준으로 OpenAI의 기본 모델은 GPT-4o이다.
Valid from: 2025-10-19 09:24:03.380744+00:00
Valid until: 2025-10-20 11:56:59.540507+00:00
--------------------------------------------------
UUID: c4478d40-fd0b-4b3c-b12d-44f05a342873
Fact: OpenAI's official website is https://openai.com/.
Valid from: 2025-10-19 09

### (2) 중심 노드 기반 검색

In [ ]:
query = "OpenAI에 대해서 알려주세요."

# 상위 검색 결과의 소스 노드 UUID를 중심 노드로 사용하여 검색합니다.
center_node_uuid = results[0].source_node_uuid

reranked_results = await graphiti.search(query, center_node_uuid)

for result in reranked_results:
    print(f"UUID: {result.uuid}")
    print(f"Fact: {result.fact}")
    if hasattr(result, "valid_at") and result.valid_at:
        print(f"Valid from: {result.valid_at}")
    if hasattr(result, "invalid_at") and result.invalid_at:
        print(f"Valid until: {result.invalid_at}")
    print("-" * 50)

2025-10-20 21:10:59 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


UUID: 19301958-398a-4976-807f-9976251958dd
Fact: GPT-5가 OpenAI의 현재 기본 모델이다.
Valid from: 2025-10-20 11:56:59.540507+00:00
--------------------------------------------------
UUID: b8eba59b-9a61-424a-afd1-7bdf8c84c760
Fact: GPT-5는 도구 없이 AIME 2025에서 94.6%를 기록했다.
--------------------------------------------------
UUID: a70c838c-d5c6-40a4-9e3b-6dd81018cace
Fact: GPT-5는 멀티모달 이해(MMU)에서 84.2%를 기록했다.
--------------------------------------------------
UUID: a298db0c-282d-4330-a05a-eec7ba684573
Fact: GPT-5는 시각적 인식 분야에서 전반적으로 훨씬 더 스마트하며 벤치마크 성과에서 우수하다.
--------------------------------------------------
UUID: 1fbaf8f1-bdcc-435b-abef-92ded9105683
Fact: GPT-5는 에이더 폴리글롯에서 88%를 기록했다.
--------------------------------------------------
UUID: 9a5323d5-4b1c-48d3-8277-88d1067d33da
Fact: GPT-5의 성능상의 이점은 일상 사용에도 적용된다.
--------------------------------------------------
UUID: 12176a8f-c5dc-47c2-af33-b2157b4d8066
Fact: OpenAI is an AI 연구 개발 기업 (AI research and development company).
Valid from: 2025-10-19 09:23:22

### (3) 검색 레피시(NODE_HYBRID_SEARCH_RRF) 사용하여 노드 검색

`NODE_HYBRID_SEARCH_RRF`는 Graphiti에서 제공하는 15개의 사전 정의된 검색 레시피(search recipe) 중 하나로, 노드(Node)에 대한 하이브리드 검색을 RRF 재순위화(reranking)와 함께 수행합니다.

#### 주요 특징

##### 1. 하이브리드 검색 (Hybrid Search)
의미적 유사도 검색 (Semantic Search): 임베딩 기반 벡터 검색
BM25 전문 검색 (Full-text Search): 키워드 기반 검색
두 방식을 결합하여 더 정확한 검색 결과 제공

##### 2. RRF (Reciprocal Rank Fusion) 재순위화
RRF는 여러 검색 알고리즘의 결과를 결합하는 알고리즘입니다:

**작동 방식:**

- 각 검색 알고리즘(BM25, 시맨틱 검색)의 결과를 순위화
- 각 결과에 역순위 점수(1/rank) 부여
- 점수를 합산하여 최종 순위 결정

**장점:**

- 각 검색 방법의 강점을 활용
- 점수 정규화 문제 회피
- 간단하면서도 효과적

##### 3. 노드 중심 검색

엣지(관계)가 아닌 노드(엔티티) 자체를 검색하고 반환합니다.

In [ ]:
from graphiti_core.search.search_config_recipes import NODE_HYBRID_SEARCH_RRF

# 1. 기본 사용 (limit 조정)
node_search_config = NODE_HYBRID_SEARCH_RRF.model_copy(deep=True)
node_search_config.limit = 5

# 2. 검색 실행
node_search_results = await graphiti.search_(
    query="gpt-5",
    config=node_search_config,
)

# 3. 결과 처리
for node in node_search_results.nodes:
    print(f"Node UUID: {node.uuid}")
    print(f"Node Name: {node.name}")
    print(f"Summary: {node.summary}")
    print(f"Labels: {node.labels}")
    print("-" * 50)

2025-10-20 21:29:38 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Node UUID: 23ce198c-85cd-4967-a62f-d51905b48956
Node Name: GPT-5
Summary: GPT-5 became OpenAI's default model as of August 8, with superior performance in math, coding, multimodal understanding, and medical benchmarks.
Labels: ['Entity']
--------------------------------------------------
Node UUID: df493d97-a036-4322-9bf6-45f786c0fa7c
Node Name: GPT-4o
Summary: GPT-4o is OpenAI's base model as of May 2024.
Labels: ['Entity']
--------------------------------------------------
Node UUID: a4791e46-e6ee-4d2d-ae91-2c6c3c9f0d3a
Node Name: MMU
Summary: MMU의 맥락에서 GPT-5는 수학, 코딩, 시각적 인식, 의료 벤치마크에서 최고 성과를 기록했다.
Labels: ['Entity']
--------------------------------------------------
Node UUID: 3e17b8c3-0f1b-48a7-b250-711861b554b3
Node Name: 일상 사용
Summary: 일상 사용: 2024년 8월 8일부로 GPT-5 도입으로 수학·코딩·시각적 인식·의료 벤치마크에서 최고 성과를 기록한다.
Labels: ['Entity']
--------------------------------------------------
Node UUID: 1f09f7c6-c920-4e6a-a43a-a8782f705f26
Node Name: SWEBench
Summary: SWEBench’s benchmark performance 

## Graphiti 에이전트 구현하기

In [ ]:
from langchain.agent import init